In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor


# Loading Data

If using this code to import, save the dataset as a csv called "Data.csv". Upload it to your notebook and run the following code.

In [3]:
df = pd.read_csv('Data.csv')
df.head()

,index,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue,Column1
0,0,02/19/16,2016.0,February,29.0,F,United States,Washington,Accessories,Tires and Tubes,1.0,80.00,109.000000,80.0,109.0,NaN
1,1,02/20/16,2016.0,February,29.0,F,United States,Washington,Clothing,Gloves,2.0,24.50,28.500000,49.0,57.0,NaN
2,2,02/27/16,2016.0,February,29.0,F,United States,Washington,Accessories,Tires and Tubes,3.0,3.67,5.000000,11.0,15.0,NaN
3,3,03/12/16,2016.0,March,29.0,F,United States,Washington,Accessories,Tires and Tubes,2.0,87.50,116.500000,175.0,233.0,NaN
4,4,03/12/16,2016.0,March,29.0,F,United States,Washington,Accessories,Tires and Tubes,3.0,35.00,41.666667,105.0,125.0,NaN


# Data Cleaning

In [4]:
df.shape

(34867, 16)

In [5]:
#dropping index and the random column, redundant
df = df.drop(['index', 'Column1'], axis=1)

In [6]:
rows_with_nan = df[df.isna().any(axis=1)]
rows_with_nan

,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue
34866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,641.532095


In [7]:
#dropping indices that have NaN value
dropped_index =df[df.isna().any(axis=1)].index
df = df.drop(dropped_index)
df

,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue
0,02/19/16,2016.0,February,29.0,F,United States,Washington,Accessories,Tires and Tubes,1.0,80.00,109.000000,80.0,109.0
1,02/20/16,2016.0,February,29.0,F,United States,Washington,Clothing,Gloves,2.0,24.50,28.500000,49.0,57.0
2,02/27/16,2016.0,February,29.0,F,United States,Washington,Accessories,Tires and Tubes,3.0,3.67,5.000000,11.0,15.0
3,03/12/16,2016.0,March,29.0,F,United States,Washington,Accessories,Tires and Tubes,2.0,87.50,116.500000,175.0,233.0
4,03/12/16,2016.0,March,29.0,F,United States,Washington,Accessories,Tires and Tubes,3.0,35.00,41.666667,105.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34861,03/22/15,2015.0,March,38.0,M,France,Charente-Maritime,Bikes,Mountain Bikes,1.0,2049.00,1487.000000,2049.0,1487.0
34862,02/07/16,2016.0,February,38.0,M,France,Hauts de Seine,Bikes,Mountain Bikes,2.0,1160.00,985.500000,2320.0,1971.0
34863,03/13/15,2015.0,March,38.0,M,France,Hauts de Seine,Bikes,Mountain Bikes,1.0,2049.00,1583.000000,2049.0,1583.0
34864,04/05/15,2015.0,April,38.0,M,France,Hauts de Seine,Bikes,Mountain Bikes,3.0,683.00,560.666667,2049.0,1682.0


In [8]:
#changed data type of year to integer
df["Year"] = df["Year"].fillna(0).astype(int)
#Customer Age to integer
df['Customer Age'] = df['Customer Age'].astype(int)

#Quantity to integer
df['Quantity'] = df['Quantity'].astype(int)

cat_columns = ['Month', 'Customer Gender', 'Country', 'State', 'Product Category', 'Sub Category']
for col in cat_columns:
    df[col] = df[col].astype('category')

df

,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue
0,02/19/16,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,1,80.00,109.000000,80.0,109.0
1,02/20/16,2016,February,29,F,United States,Washington,Clothing,Gloves,2,24.50,28.500000,49.0,57.0
2,02/27/16,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,3,3.67,5.000000,11.0,15.0
3,03/12/16,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,2,87.50,116.500000,175.0,233.0
4,03/12/16,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,3,35.00,41.666667,105.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34861,03/22/15,2015,March,38,M,France,Charente-Maritime,Bikes,Mountain Bikes,1,2049.00,1487.000000,2049.0,1487.0
34862,02/07/16,2016,February,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,2,1160.00,985.500000,2320.0,1971.0
34863,03/13/15,2015,March,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,1,2049.00,1583.000000,2049.0,1583.0
34864,04/05/15,2015,April,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,3,683.00,560.666667,2049.0,1682.0


In [9]:
#Changing Date to Datetime data type
df["Date"] = pd.to_datetime(df["Date"])
df

/var/folders/__/lgxf7nwn2rjgty_h2t6705840000gn/T/ipykernel_4138/1788840353.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])


,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue
0,2016-02-19,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,1,80.00,109.000000,80.0,109.0
1,2016-02-20,2016,February,29,F,United States,Washington,Clothing,Gloves,2,24.50,28.500000,49.0,57.0
2,2016-02-27,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,3,3.67,5.000000,11.0,15.0
3,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,2,87.50,116.500000,175.0,233.0
4,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,3,35.00,41.666667,105.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34861,2015-03-22,2015,March,38,M,France,Charente-Maritime,Bikes,Mountain Bikes,1,2049.00,1487.000000,2049.0,1487.0
34862,2016-02-07,2016,February,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,2,1160.00,985.500000,2320.0,1971.0
34863,2015-03-13,2015,March,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,1,2049.00,1583.000000,2049.0,1583.0
34864,2015-04-05,2015,April,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,3,683.00,560.666667,2049.0,1682.0


# Feature Engineering

In [10]:
# Adding profit column
df['Profit'] = df['Revenue'] - df['Cost']
df

,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue,Profit
0,2016-02-19,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,1,80.00,109.000000,80.0,109.0,29.0
1,2016-02-20,2016,February,29,F,United States,Washington,Clothing,Gloves,2,24.50,28.500000,49.0,57.0,8.0
2,2016-02-27,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,3,3.67,5.000000,11.0,15.0,4.0
3,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,2,87.50,116.500000,175.0,233.0,58.0
4,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,3,35.00,41.666667,105.0,125.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34861,2015-03-22,2015,March,38,M,France,Charente-Maritime,Bikes,Mountain Bikes,1,2049.00,1487.000000,2049.0,1487.0,-562.0
34862,2016-02-07,2016,February,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,2,1160.00,985.500000,2320.0,1971.0,-349.0
34863,2015-03-13,2015,March,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,1,2049.00,1583.000000,2049.0,1583.0,-466.0
34864,2015-04-05,2015,April,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,3,683.00,560.666667,2049.0,1682.0,-367.0


In [11]:
#Adding profit margin column
df["Profit Margin"] = df["Profit"] / df["Revenue"]
df

,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue,Profit,Profit Margin
0,2016-02-19,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,1,80.00,109.000000,80.0,109.0,29.0,0.266055
1,2016-02-20,2016,February,29,F,United States,Washington,Clothing,Gloves,2,24.50,28.500000,49.0,57.0,8.0,0.140351
2,2016-02-27,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,3,3.67,5.000000,11.0,15.0,4.0,0.266667
3,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,2,87.50,116.500000,175.0,233.0,58.0,0.248927
4,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,3,35.00,41.666667,105.0,125.0,20.0,0.160000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34861,2015-03-22,2015,March,38,M,France,Charente-Maritime,Bikes,Mountain Bikes,1,2049.00,1487.000000,2049.0,1487.0,-562.0,-0.377942
34862,2016-02-07,2016,February,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,2,1160.00,985.500000,2320.0,1971.0,-349.0,-0.177067
34863,2015-03-13,2015,March,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,1,2049.00,1583.000000,2049.0,1583.0,-466.0,-0.294378
34864,2015-04-05,2015,April,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,3,683.00,560.666667,2049.0,1682.0,-367.0,-0.218193


In [12]:
age_bins = [0, 18, 25, 35, 45, 55, 65, 100]
age_labels = ['<18', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']

# Add a new column with the binned ages
df['Age Group'] = pd.cut(df['Customer Age'], bins=age_bins, labels=age_labels, right=False)
df

,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue,Profit,Profit Margin,Age Group
0,2016-02-19,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,1,80.00,109.000000,80.0,109.0,29.0,0.266055,25-34
1,2016-02-20,2016,February,29,F,United States,Washington,Clothing,Gloves,2,24.50,28.500000,49.0,57.0,8.0,0.140351,25-34
2,2016-02-27,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,3,3.67,5.000000,11.0,15.0,4.0,0.266667,25-34
3,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,2,87.50,116.500000,175.0,233.0,58.0,0.248927,25-34
4,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,3,35.00,41.666667,105.0,125.0,20.0,0.160000,25-34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34861,2015-03-22,2015,March,38,M,France,Charente-Maritime,Bikes,Mountain Bikes,1,2049.00,1487.000000,2049.0,1487.0,-562.0,-0.377942,35-44
34862,2016-02-07,2016,February,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,2,1160.00,985.500000,2320.0,1971.0,-349.0,-0.177067,35-44
34863,2015-03-13,2015,March,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,1,2049.00,1583.000000,2049.0,1583.0,-466.0,-0.294378,35-44
34864,2015-04-05,2015,April,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,3,683.00,560.666667,2049.0,1682.0,-367.0,-0.218193,35-44


In [13]:
df['Numerical Month'] = df['Date'].dt.month

# Define a function to map months to fiscal quarters
def get_quarter(month):
    if month in [3, 1, 2]:
        return 'Q1'
    elif month in [6, 4, 5]:
        return 'Q2'
    elif month in [9, 7, 8]:
        return 'Q3'
    elif month in [12, 10, 11]:
        return 'Q4'
    else:
        return None

# Apply the function to create a new 'Season' column
df['Quarter'] = df['Numerical Month'].apply(get_quarter)
df

,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue,Profit,Profit Margin,Age Group,Numerical Month,Quarter
0,2016-02-19,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,1,80.00,109.000000,80.0,109.0,29.0,0.266055,25-34,2,Q1
1,2016-02-20,2016,February,29,F,United States,Washington,Clothing,Gloves,2,24.50,28.500000,49.0,57.0,8.0,0.140351,25-34,2,Q1
2,2016-02-27,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,3,3.67,5.000000,11.0,15.0,4.0,0.266667,25-34,2,Q1
3,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,2,87.50,116.500000,175.0,233.0,58.0,0.248927,25-34,3,Q1
4,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,3,35.00,41.666667,105.0,125.0,20.0,0.160000,25-34,3,Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34861,2015-03-22,2015,March,38,M,France,Charente-Maritime,Bikes,Mountain Bikes,1,2049.00,1487.000000,2049.0,1487.0,-562.0,-0.377942,35-44,3,Q1
34862,2016-02-07,2016,February,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,2,1160.00,985.500000,2320.0,1971.0,-349.0,-0.177067,35-44,2,Q1
34863,2015-03-13,2015,March,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,1,2049.00,1583.000000,2049.0,1583.0,-466.0,-0.294378,35-44,3,Q1
34864,2015-04-05,2015,April,38,M,France,Hauts de Seine,Bikes,Mountain Bikes,3,683.00,560.666667,2049.0,1682.0,-367.0,-0.218193,35-44,4,Q2


In [14]:
df.dtypes

Date                datetime64[ns]
Year                         int64
Month                     category
Customer Age                 int64
Customer Gender           category
Country                   category
State                     category
Product Category          category
Sub Category              category
Quantity                     int64
Unit Cost                  float64
Unit Price                 float64
Cost                       float64
Revenue                    float64
Profit                     float64
Profit Margin              float64
Age Group                 category
Numerical Month              int32
Quarter                     object
dtype: object

**KNN Regression Technique: Predicting Product Quantity Based on Customer Age, Gender, and Country**

In [41]:
#features
X = df[['Customer Age', 'Customer Gender', 'Country']]
#target variable
y = df['Quantity']

#encoding categorical columns
X = pd.get_dummies(X)

#Normalize data 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [42]:
from sklearn.model_selection import train_test_split

#train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 42)

#train KNN
knn = KNeighborsRegressor(n_neighbors = 4)
knn.fit(X_train, y_train)

#predict
y_pred = knn.predict(X_test)

In [43]:
from sklearn.metrics import mean_squared_error, r2_score
print("RMSE:", mean_squared_error(y_test, y_pred))
print("r2:", r2_score(y_test, y_pred))

RMSE: 0.8339905362776026
r2: -0.26495278427574576


The RMSE and r2 are not great which means that the KNN model is not able to see patterns in the data. This could mean that the features I selected are not helping the model make good predictions.

**KNN Classification**

In [44]:
X = df[['Customer Age', 'Customer Gender', 'Country']]
X = pd.get_dummies(X)

y = df['Quantity']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 42)

In [45]:
#training KNN classifer
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 4)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

In [46]:
from sklearn.metrics import accuracy_score, f1_score
print("Accuracy:", accuracy_score(y_test, y_pred))
print("f1", f1_score(y_test, y_pred, average = 'weighted'))

Accuracy: 0.3350860420650096
f1 0.33464735322281003


The accuracy and f1 score are pretty low which might suggest that the features I selected might not be sufficient information to make this prediction. It could also mean that KNN is not a good model to make this type of prediction.